In [ ]:
%matplotlib inline

import ROOT, sys, os
from ROOT import std

from larcv import larcv
from larlite import larlite as ll
from larlite import larutil as lu

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.patches as patches

from ROOT import geo2d,cv
from ROOT.cv import Point_ as Vector
DTYPE='float'

import root_numpy as rn
import pandas as pd

geoh=lu.GeometryHelper.GetME()
geo=lu.Geometry.GetME()
larp=lu.LArProperties.GetME()
pygeo = geo2d.PyDraw()

matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'

In [ ]:
reco_df = pd.DataFrame(rn.root2rec("../reco_ana.root",treename='LArbysImageTree'))
mc_df   = pd.DataFrame(rn.root2rec("../mc_ana.root",treename='mctree'))

In [ ]:
#Calculating the signal denomenator...
pdg2mass={}
pdg2mass[13]  = 105
pdg2mass[2212]= 938
pdg2mass[2112]= 939.5
pdg2mass[11]  = 0#0.5
pdg2mass[311] = 497.65
def mass(pdg):
    return pdg2mass[pdg]
mass_vec=np.vectorize(mass)

mc_wdf=mc_df.copy()
print mc_wdf.index.size
mc_wdf=mc_wdf.query('energyInit > 100 & energyInit < 500')
print mc_wdf.index.size
mc_wdf=mc_wdf[mc_wdf.daughterPdg_v.apply(lambda x : x.size>=2)]
print mc_wdf.index.size

def containment(row):
    pdg_v   = row['daughterPdg_v']
    einit_v = row['daughterEnergyInit_v']
    edep_v  = row['daughterEnergyDep_v']
    
    eisub_v=einit_v-mass_vec(pdg_v)
  
    init_sum = eisub_v.sum()
    dep_sum  = edep_v.sum()
    
    proton_v=eisub_v[np.where(pdg_v==2212)[0]]
    
    if(np.where(proton_v>80)[0].size>0): pass
    else: return False;
        
    if dep_sum / init_sum < 0.9: return False;

    return True;
    
mc_wdf=mc_wdf[mc_wdf.apply(containment,axis=1)]
print mc_wdf.index.size
mc_wdf_idx = mc_wdf.index

In [ ]:
#Calculating the efficiency numerator
#reco_df_copy will have out analysis cuts built in
reco_wdf = reco_df.ix[mc_wdf_idx].copy()

cuts_v=[]
cuts_v.append('n_vtx3d>0')

for cut in cuts_v:
    reco_wdf = reco_wdf.query(cut)

print reco_wdf.index.size

#filter vertices from, only care about the events that have
#vertex estimated from 3 planes
#reco_wdf = reco_wdf[reco_wdf['num_planes_v'].apply(lambda x : np.where(x==3)[0].size)>0]

print reco_wdf.index.size

#cut on the pixel frac
#pxfrac=0.9
#def pixels(x):
#    y=np.hstack(x)
#    return np.where(y>pxfrac)[0].size

#reco_wdf=reco_wdf[reco_wdf['num_pixel_frac_vv'].apply(pixels) > 0]

In [ ]:
efficiency = 96/112.
print efficiency

In [ ]:
#2d resolution

def hi_pix_vtx_x(row):
    vtx_x_vv        = row['circle_vtx_x_vv']
    num_pix_frac_vv = row['num_pixel_frac_vv']
    idx=np.argmax(np.sum(np.vstack(num_pix_frac_vv),axis=1))
    return list(vtx_x_vv[idx])

def hi_pix_vtx_y(row):
    vtx_y_vv        = row['circle_vtx_y_vv']
    num_pix_frac_vv = row['num_pixel_frac_vv']
    idx=np.argmax(np.sum(np.vstack(num_pix_frac_vv),axis=1))
    return list(vtx_y_vv[idx])


reco_w=reco_wdf.apply(hi_pix_vtx_x,axis=1).values
reco_t=reco_wdf.apply(hi_pix_vtx_y,axis=1).values


dw_v=mc_df.ix[reco_wdf.index].vtx2d_w.values-reco_t
dt_v=mc_df.ix[reco_wdf.index].vtx2d_t.values-reco_w


dw_v=np.vstack(dw_v)
dt_v=np.vstack(dt_v)

In [ ]:
#2d resolution plots 

fig,ax=plt.subplots(figsize=(10,6))
for i in xrange(3):
    ax.hist(dw_v[:,i],
            label='Plane {}'.format(i),
            bins=np.arange(-80,80+2,2),
            alpha=0.5)
ax.set_xlabel("(MC - Reco) Wire [pixel]",fontweight='bold')
ax.set_ylim(0,45)
ax.set_xlim(-100,100)
ax.legend()
plt.grid()
plt.show()

fig,ax=plt.subplots(figsize=(10,6))
for i in xrange(3):
    ax.hist(dt_v[:,i],
            label='Plane {}'.format(i),
            bins=np.arange(-80,80+2,2),
            alpha=0.5)
ax.set_xlabel("(MC - Reco) Time [pixel]",fontweight='bold')
ax.set_ylim(0,45)
ax.set_xlim(-100,100)
ax.legend()
plt.grid()
plt.show()


for i in xrange(3):
    fig,ax=plt.subplots(figsize=(10,6))
    ax.hist(np.sqrt(np.power(dw_v[:,i],2)+np.power(dt_v[:,i],2)),
            label='Plane {}'.format(i),
            bins=np.arange(0,100+1,1),
            alpha=0.5,lw=2)
    ax.set_xlabel("Dist MC Reco Vtx [pixel]",fontweight='bold')
    ax.set_ylim(0,30)
    ax.set_xlim(0,100)
    ax.legend()
    plt.grid()
    plt.show()

In [ ]:
#num_pixel_frac check
#2d resolution

def hi_pix_v(row):
    num_pix_frac_vv = row['num_pixel_frac_vv']
    idx=np.argmax(np.sum(np.vstack(num_pix_frac_vv),axis=1))
    return list(num_pix_frac_vv[idx])



p_=reco_wdf.apply(hi_pix_v,axis=1).values
p_=np.vstack(p_)
for i in xrange(3):
    fig,ax=plt.subplots(figsize=(10,6))
    ax.hist(p_[:,i],bins=np.arange(0,1+0.6,0.1))
    ax.set_xlabel("Plane {}".format(i))
    ax.set_ylim(0,45)
    ax.set_xlim(0,1.5)
    plt.grid()
    plt.show()

fig,ax=plt.subplots(figsize=(10,6))
ax.hist(p_.sum(axis=1),bins=np.arange(0,4+0.1,0.1))
ax.set_xlabel("Sum")
plt.grid()
plt.show()

fig,ax=plt.subplots(figsize=(10,6))
ppp=p_[:,0]*p_[:,1]*p_[:,2]
ax.hist(ppp,bins=np.arange(0,1+0.6,0.1))
ax.set_xlim(0,1.0)
ax.set_xlabel("Product")
plt.grid()
plt.show()